In [ ]:
import os
import random
#from lxml import etree
from queue import Queue

from tqdm import tqdm
import threading
from src.utils import *

# -*- coding: utf-8 -*-
import time
from openai import OpenAI
import openai
client=OpenAI(api_key='<KEY>')
client.api_key=''
client.base_url=''
POOL_DATA_PATH = "./results/"

In [ ]:

import numpy as np
import networkx as nx


def select_balance_outcomes(cases, number, self_assistant="[[A]]"):
    texts = [_["judgment"] for _ in cases]
    self_group = []
    other_group = []
    for text, case in zip(texts, cases):
        if self_assistant in text:
            self_group.append(case)
        else:
            other_group.append(case)
    random.shuffle(self_group)
    random.shuffle(other_group)
    select_group = self_group[:number-int(number/2)]+other_group[:int(number/2)]
    return select_group


def select_self_outcomes(cases, number, self_assistant="[[A]]"):
    texts = [_["judgment"] for _ in cases]
    self_group = []
    other_group = []
    for text, case in zip(texts, cases):
        if self_assistant in text:
            self_group.append(case)
        else:
            other_group.append(case)
    random.shuffle(self_group)
    random.shuffle(other_group)
    select_group = self_group[:number]
    return select_group


def select_other_outcomes(cases, number, self_assistant="[[A]]"):
    texts = [_["judgment"] for _ in cases]
    self_group = []
    other_group = []
    for text, case in zip(texts, cases):
        if self_assistant in text:
            self_group.append(case)
        else:
            other_group.append(case)
    random.shuffle(self_group)
    random.shuffle(other_group)
    select_group = other_group[:number]
    return select_group

def select_right_outcomes(cases, number, self_assistant="[[A]]"):
    texts = [_["judgment"] for _ in cases]
    self_group = []
    other_group = []
    for text, case in zip(texts, cases):
        if self_assistant in text:
            self_group.append(case)
        else:
            other_group.append(case)
    random.shuffle(self_group)
    random.shuffle(other_group)
    select_group = other_group[:number]
    return select_group
def select_wrong_outcomes(cases, number, self_assistant="[[A]]"):
    texts = [_["judgment"] for _ in cases]
    self_group = []
    other_group = []
    for text, case in zip(texts, cases):
        if self_assistant in text:
            self_group.append(case)
        else:
            other_group.append(case)
    random.shuffle(self_group)
    random.shuffle(other_group)
    select_group = other_group[:number]
    return select_group

In [ ]:
#那我们要read pool的数据，然后
benchmark="evalbias"
prompt_template="mtbench"
base_judge="gpt-4o"
budget = 16
benchmark_data_with_pool = read_json(POOL_DATA_PATH+benchmark+"_"+prompt_template+"_PoolModified_"+base_judge+".json")

In [ ]:
#random.seed(7)

ALPHABAT = "CDEFGHIJKLMNOPQRSTUVWXYZ"

mtbench_template = """
Please act as an impartial and comprehensive judge evaluating the quality of two AI-generated responses (Assistant A and Assistant B) to the user's question. Your role is to determine which assistant better fulfills the user's needs according to multiple criteria, without focusing narrowly on any single aspect.

[Related Judgments - Internal Reference Only]: {previous_evaluation}.
These judgments are strictly internal; do not quote or refer to them. They are only to ensure you apply a consistent and comprehensive standard.

Evaluation Principles:
1. Holistic Assessment: Consider a wide range of factors—helpfulness, relevance, accuracy, completeness, clarity, depth, reasoning quality, creativity, level of detail—and maintain balance among these factors.
2. Comparative Contextualization: Although your final evaluation should be based solely on the two given responses, you may imagine or recall characteristics of other responses you've encountered in similar situations as background context. Implicitly inspire from the specific details in these related judgments for your current evaluation and refine the evaluation by identifying any missing dimensions or subpoints that are relevant for comparing Assistant A and Assistant B. Use this mental benchmark to ensure you apply consistent standards. Do not, however, introduce new information from outside sources into the final judgment. The goal is to remain aware of what an ideal response might look like, to ensure neither given answer is judged too narrowly.
3. Fairness and Impartiality: Do not be influenced by length, style, order of appearance, or assistant names. Evaluate purely on content quality.
4. Avoid External Biases: Rely only on the two given responses. Do not bring in outside data.

Instructions:
1. Please read the above related judgments carefully. Do not include the content under [Related Judgments] in your final output, nor should you explicitly reference them. These related judgments are for your internal reasoning only, serving as a mental benchmark to guide a comprehensive and fair evaluation. Use them as reference points to identify helpful aspects, potential weaknesses, or key criteria missed in prior assessments. Do not mention or hint at their existence in the final answer.
2. Evaluate both Assistant A’s and Assistant B’s answers thoroughly using the criteria above.
3. Compare their strengths and weaknesses, guided by your internal standard. Again, do not mention or allude to the internal references.
4. Begin your evaluation by comparing the two responses and provide a explanation.
5. End with your final verdict on which assistant performed better using the exact format:
   - [[A]] if Assistant A is better.
   - [[B]] if Assistant B is better.


[User Question]
{question}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]
"""


mtbench_multiturn_template = """
Please act as an impartial and comprehensive judge evaluating the quality of two AI-generated responses (Assistant A and Assistant B) to the user's question. Your role is to determine which assistant better fulfills the user's needs according to multiple criteria, without focusing narrowly on any single aspect. You should focus on who provides a better answer to the second user question.

[Related Judgments - Internal Reference Only]: {previous_evaluation}.
These judgments are strictly internal; do not quote or refer to them. They are only to ensure you apply a consistent and comprehensive standard.

Evaluation Principles:
1. Holistic Assessment: Consider a wide range of factors—helpfulness, relevance, accuracy, completeness, clarity, depth, reasoning quality, creativity, level of detail—and maintain balance among these factors.
2. Comparative Contextualization: Although your final evaluation should be based solely on the two given responses, you may imagine or recall characteristics of other judgments you've encountered in similar situations as background context. Implicitly inspire from the specific details in these related judgments for your current evaluation and refine the evaluation by identifying any missing dimensions or subpoints that are relevant for comparing Assistant A and Assistant B. Use this mental benchmark to ensure you apply consistent standards. Do not, however, introduce new information from outside sources into the final judgment. The goal is to remain aware of what an ideal response might look like, to ensure neither given answer is judged too narrowly.
3. Fairness and Impartiality: Do not be influenced by length, style, order of appearance, or assistant names. Evaluate purely on content quality.
4. Avoid External Biases: Rely only on the two given responses. Do not bring in outside data.

Instructions:
1. Please read the above related judgments carefully. Do not include the content under [Related Judgments] in your final output, nor should you explicitly reference them. These related judgments are for your internal reasoning only, serving as a mental benchmark to guide a comprehensive and fair evaluation. Use them as reference points to identify helpful aspects, potential weaknesses, or key criteria missed in prior assessments. Do not mention or hint at their existence in the final answer.
2. Evaluate both Assistant A’s and Assistant B’s answers thoroughly using the criteria above, paying attention to the second question's answers.
3. Compare their strengths and weaknesses, guided by your internal standard. Again, do not mention or allude to the internal references.
4. Begin your evaluation by comparing the two responses and provide a explanation.
5. End with your final verdict on which assistant performed better using the exact format:
   - [[A]] if Assistant A is better.
   - [[B]] if Assistant B is better.
   - Omit any other output.


<|The Start of Assistant A's Conversation with User|>

### User:
{question_1}

### Assistant A:
{answer_a_1}

### User:
{question_2}

### Assistant A:
{answer_a_2}

<|The End of Assistant A's Conversation with User|>


<|The Start of Assistant B's Conversation with User|>

### User:
{question_1}

### Assistant B:
{answer_b_1}

### User:
{question_2}

### Assistant B:
{answer_b_2}

<|The End of Assistant B's Conversation with User|>
"""
#helpsteer2_template = (
#    "In your assessment, please take into account previously provided judgments to enhance your evaluation. Use them as reference points to identify helpful aspects, potential weaknesses, or key criteria missed in prior assessments.\n"
#    "[Previous Evaluations for Reference] "
#    "{previous_evaluation} "
#    "[Current Task]"
#    "Now, you are asked to evaluate the responses of Assistant A and Assistant B to the user question below."
#    "You should choose the assistant that better follows the user's instructions and answers the user's question. Your evaluation should consider factors such as helpfulness, correctness, coherence, complexity, verbosity."
#    "Refer back to the previous evaluations for insights on how each assistant performed on similar criteria in past responses, but ensure that your current evaluation remains unbiased and grounded in the responses at hand.\n"
#    "Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision."
#    "Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants."
#    "Be as objective as possible."
#    "After providing your explanation, output your final verdict by strictly following this format: '[[A]]' if Assistant A is better, or '[[B]]' if Assistant B is better."
#    "[User Question]\n{question}\n\n[The Start of Assistant A's Answer]\n{answer_a}\n[The End of Assistant A's Answer]\n\n[The Start of Assistant B's Answer]\n{answer_b}\n[The End of Assistant B's Answer]"
#)

#helpsteer2_template = (
#    "Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants, A and B, to the user question displayed below. Your task is to decide which assistant follows the user's instructions and answers the question better. To enhance your evaluation, you have access to previous judgments derived from comparisons between Assistants A/B and other AI assistants. "
#    "[Context Judgments Information]\n{previous_evaluation}\n"
#    "Use these context judgments as a supplementary tool but not as a rigid standard; instead, consider them contextually and maintain an independent assessment of A and B's responses. "
#    "Your evaluation should consider factors such as helpfulness, correctness, coherence, complexity, verbosity. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: '[[A]]' if assistant A is better, '[[B]]' if assistant B is better.\n "
#    "[User Question]\n{question}\n\n[The Start of Assistant A's Answer]\n{answer_a}\n[The End of Assistant A's Answer]\n\n[The Start of Assistant B's Answer]\n{answer_b}\n[The End of Assistant B's Answer]\n"
#)

helpsteer2_template = """
Please act as an impartial and comprehensive judge evaluating the quality of two AI-generated responses (Assistant A and Assistant B) to the user's question. Your role is to determine which assistant better fulfills the user's needs according to multiple criteria, without focusing narrowly on any single aspect.

[Related Judgments - Internal Reference Only]: {previous_evaluation}.
These judgments are strictly internal; do not quote or refer to them. They are only to ensure you apply a consistent and comprehensive standard.

Evaluation Principles:
1. Holistic Assessment: Consider a wide range of factors—helpfulness, correctness, coherence, complexity, verbosity to user instructions—and maintain balance among these factors.
2. Comparative Contextualization: Although your final evaluation should be based solely on the two given responses, you may imagine or recall characteristics of other responses you've encountered in similar situations as background context. Implicitly inspire from the specific details in these related judgments for your current evaluation and refine the evaluation by identifying any missing dimensions or subpoints that are relevant for comparing Assistant A and Assistant B. Use this mental benchmark to ensure you apply consistent standards. Do not, however, introduce new information from outside sources into the final judgment. The goal is to remain aware of what an ideal response might look like, to ensure neither given answer is judged too narrowly.
3. Fairness and Impartiality: Do not be influenced by length, style, order of appearance, or assistant names. Evaluate purely on content quality.
4. Avoid External Biases: Rely only on the two given responses. Do not bring in outside data.

Instructions:
1. Please read the above related judgments carefully. Do not include the content under [Related Judgments] in your final output, nor should you explicitly reference them. These related judgments are for your internal reasoning only, serving as a mental benchmark to guide a comprehensive and fair evaluation. Use them as reference points to identify helpful aspects, potential weaknesses, or key criteria missed in prior assessments. Do not mention or hint at their existence in the final answer.
2. Evaluate both Assistant A’s and Assistant B’s answers thoroughly using the criteria above.
3. Compare their strengths and weaknesses, guided by your internal standard. Again, do not mention or allude to the internal references.
4. Begin your evaluation by comparing the two responses and provide a explanation.
5. End with your final verdict on  which assistant performed better by strictly following this format:
   - [[A]] if Assistant A is better.
   - [[B]] if Assistant B is better.
   - Omit any other output.


[User Question]
{question}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]
"""

def custom_previous_evaluation(case, budget):
    
    pool_a = case["external_pool"]["position_a"]
    pool_b = case["external_pool"]["position_b"]
    random.shuffle(pool_a)
    random.shuffle(pool_b)
    a_length = int(budget/2)
    #print(a_length)
    b_length = budget-a_length

    pool_a = select_other_outcomes(pool_a, a_length, "[[B]]")
    pool_b = select_other_outcomes(pool_b, b_length, "[[A]]")
    pool_a = pool_a[:a_length]
    pool_b = pool_b[:b_length]
    assistant_dict = {}
    for _,Character in zip(pool_a,ALPHABAT):
        assistant_dict[_["model_name"]]=Character
    for _,Character in zip(pool_b,ALPHABAT[len(pool_a):]):
        if _["model_name"] not in assistant_dict:
            assistant_dict[_["model_name"]]=Character
    COUNT = 1
    reference_segments = ""
    
    for instance in pool_b:
        reference = str(COUNT)+". Evaluation between Assistant A and Assistant "+assistant_dict[instance["model_name"]]+":"
        reference += instance["judgment"].replace("Assistant B", "Assistant "+assistant_dict[instance["model_name"]]).replace("[[B]]", "[["+assistant_dict[instance["model_name"]]+"]]").replace("Response B", "Assistant "+assistant_dict[instance["model_name"]])
        reference += "\n"
        reference_segments+=reference
        COUNT+=1
    for instance in pool_a:
        reference = str(COUNT)+". Evaluation between Assistant "+assistant_dict[instance["model_name"]]+" and Assistant B:"
        reference += instance["judgment"].replace("Assistant A", "Assistant "+assistant_dict[instance["model_name"]]).replace("[[A]]", "[["+assistant_dict[instance["model_name"]]+"]]").replace("Response A", "Assistant "+assistant_dict[instance["model_name"]])
        reference += "\n"
        reference_segments+=reference
        COUNT+=1

    
    return reference_segments
#parse to prompt, 这里要做的是，根据输入budget，随机从pool里选择
def parse_to_prompts(prompt_template, benchmark_data_with_pool, budget):
    
    thread_inputs = []
    for case in tqdm(benchmark_data_with_pool):
        previous_evaluation = custom_previous_evaluation(case, budget)
        
        system_prompt = None
        if prompt_template == "mtbench":
            if not case["multiturn"]:
                gpt_input = mtbench_template.format(
                    previous_evaluation=previous_evaluation,
                    question=case["instruction"],
                    answer_a=case["answer_a"][1]["content"],
                    answer_b=case["answer_b"][1]["content"],
                )
                system_prompt = {
                    "role": "system", 
                    "content": "You are an impartial and comprehensive judge. Your task is to compare the responses of Assistant A and Assistant B to the user question provided by the user. To do this effectively, you will: 1. Review the detailed evaluation results from previous comparisons of Assistant AB with other Assistants (e.g., C, D, E). These evaluations include key dimensions such as helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. 2. Extract the core dimensions and specific details from the initial evaluations to establish a baseline for your current evaluation.\n 3. Dynamically expand the evaluation by identifying any missing dimensions or subpoints that are relevant for comparing Assistant A and Assistant B. Justify the inclusion of these new dimensions.\n 4.Perform a thorough comparison of Assistant A and Assistant B using both inherited and expanded dimensions, ensuring objectivity and fairness.\n You must avoid position or naming biases, and the length of the responses should not influence your evaluation. **Output Requirements:** 1. List inherited dimensions and their sources, followed by newly expanded dimensions and their justifications. 2. Provide a detailed comparison of Assistant A and Assistant B under each dimension, highlighting their strengths and weaknesses.3. Conclude with your verdict in the following strict format: [[A]] if Assistant A is better overall. [[B]] if Assistant B is better overall."
                }
                #system_prompt={"role": "system", "content": "Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. Your task is to decide which assistant follows the user's instructions and answers the question better, considering relevant evaluation examples. "}
            else:
                #system_prompt = mtbench_multiturn_template["system_prompt"]
                gpt_input = mtbench_multiturn_template.format(
                    previous_evaluation=previous_evaluation,
                    question_1=case["answer_a"][0]["content"],
                    question_2=case["answer_a"][2]["content"],
                    answer_a_1=case["answer_a"][1]["content"],
                    answer_b_1=case["answer_b"][1]["content"],
                    answer_a_2=case["answer_a"][3]["content"],
                    answer_b_2=case["answer_b"][3]["content"],
                )
                #system_prompt={"role": "system", "content": "Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below, considering relevant evaluation examples. "}
                system_prompt = {
                    "role": "system", 
                    "content": "You are an impartial and comprehensive judge. Your task is to compare the responses of Assistant A and Assistant B to the user question provided by the user. You should focus on who provides a better answer to the second user question. To do this effectively, you will: 1. Review the detailed evaluation results from previous comparisons of Assistant AB with other Assistants (e.g., C, D, E). These evaluations include key dimensions such as helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. 2. Extract the core dimensions and specific details from the initial evaluations to establish a baseline for your current evaluation.\n 3. Dynamically expand the evaluation by identifying any missing dimensions or subpoints that are relevant for comparing Assistant A and Assistant B. Justify the inclusion of these new dimensions.\n 4.Perform a thorough comparison of Assistant A and Assistant B using both inherited and expanded dimensions, ensuring objectivity and fairness.\n You must avoid position or naming biases, and the length of the responses should not influence your evaluation. **Output Requirements:** 1. List inherited dimensions and their sources, followed by newly expanded dimensions and their justifications. 2. Provide a detailed comparison of Assistant A and Assistant B under each dimension, highlighting their strengths and weaknesses.3. Conclude with your verdict in the following strict format: [[A]] if Assistant A is better overall. [[B]] if Assistant B is better overall."
                }
        elif prompt_template == "helpsteer2":
            gpt_input = helpsteer2_template.format(
                previous_evaluation=previous_evaluation,
                question=case["instruction"],
                answer_a=case["answer_a"][1]["content"],
                answer_b=case["answer_b"][1]["content"],
            )
            system_prompt = {
                "role": "system", 
                "content": "You are an impartial and comprehensive evaluator. Your task is to compare the responses of Assistant A and Assistant B to the user question provided by the user. To do this effectively, you will: 1. Review the detailed evaluation results from previous comparisons of Assistant AB with other Assistants (e.g., C, D, E). These evaluations include key dimensions such as helpfulness, correctness, coherence, complexity, verbosity to user instructions. 2. Extract the core dimensions and specific details from the initial evaluations to establish a baseline for your current evaluation.\n 3. Dynamically expand the evaluation by identifying any missing dimensions or subpoints that are relevant for comparing Assistant A and Assistant B. Justify the inclusion of these new dimensions.\n 4.Perform a thorough comparison of Assistant A and Assistant B using both inherited and expanded dimensions, ensuring objectivity and fairness.\n You must avoid position or naming biases, and the length of the responses should not influence your evaluation. **Output Requirements:** 1. List inherited dimensions and their sources, followed by newly expanded dimensions and their justifications. 2. Provide a detailed comparison of Assistant A and Assistant B under each dimension, highlighting their strengths and weaknesses.3. Conclude with your verdict in the following strict format: [[A]] if Assistant A is better overall. [[B]] if Assistant B is better overall."
            }
        elif prompt_template == "lfqa":
            gpt_input = lfqa_template.format(
                previous_evaluation=previous_evaluation,
                question=case["instruction"],
                answer_a=case["answer_a"][1]["content"],
                answer_b=case["answer_b"][1]["content"],
            )
            system_prompt = {
                "role": "system", 
                "content": "You are an impartial and comprehensive evaluator. Your task is to compare the responses of Assistant A and Assistant B to the user question provided by the user. To do this effectively, you will: 1. Review the detailed evaluation results from previous comparisons of Assistant AB with other Assistants (e.g., C, D, E). These evaluations include key dimensions such as i) the response should consider all aspects of the question and be well formulated and easy to follow; ii). the response should not contain irrelevant information or factually incorrect information or common misconceptions. 2. Extract the core dimensions and specific details from the initial evaluations to establish a baseline for your current evaluation.\n 3. Dynamically expand the evaluation by identifying any missing dimensions or subpoints that are relevant for comparing Assistant A and Assistant B. Justify the inclusion of these new dimensions.\n 4.Perform a thorough comparison of Assistant A and Assistant B using both inherited and expanded dimensions, ensuring objectivity and fairness.\n You must avoid position or naming biases, and the length of the responses should not influence your evaluation. **Output Requirements:** 1. List inherited dimensions and their sources, followed by newly expanded dimensions and their justifications. 2. Provide a detailed comparison of Assistant A and Assistant B under each dimension, highlighting their strengths and weaknesses.3. Conclude with your verdict in the following strict format: [[A]] if Assistant A is better overall. [[B]] if Assistant B is better overall."
            }
            #system_prompt = {"role": "system", "content": "You are a helpful assistant in evaluating the quality of the responses for a given instruction. The responses being evaluated are likely longer form responses to questions requiring in-depth reasoning."}
        elif prompt_template == "judgebench":
            gpt_input = judgebench_template.format(
                previous_evaluation=previous_evaluation,
                question=case["instruction"],
                answer_a=case["answer_a"][1]["content"],
                answer_b=case["answer_b"][1]["content"],
            )
        thread_inputs.append([{"role": "user", "content": gpt_input}])
    return thread_inputs
prompts = parse_to_prompts(prompt_template, benchmark_data_with_pool, budget=budget)

100%|██████████| 519/519 [00:00<00:00, 15099.56it/s]


In [253]:
def api(thread_inputs):
    class Crawl_thread(threading.Thread):
        def __init__(self, thread_id, queue):
            threading.Thread.__init__(self)
            self.thread_id = thread_id
            #self.filename = target.replace("*", "_thread" + str(self.thread_id) + ".jsonl")
            self.queue = queue
        
        def run(self):
            print("start thread:", self.thread_id)
            self.crawl_spider()
            print("quit thread:", self.thread_id)
        
        def crawl_spider(self):
            global all_get_data3, candidate_key
            while True:
                if self.queue.empty():
                    break
                else:
                    row= self.queue.get()
                    #msgs=row["prompt_new"]
                    msgs = row #self.openai_template(row[0], row[1])
                    try:
                        success = False
                        for attempt in range(5):
                            try:
                                #client.api_key = random.choice(all_keys)
                                response = client.chat.completions.create(
                                    model = "gpt-4o",
                                    messages=msgs, 
                                    temperature=0.1
                                )
                            except Exception as e:
                                time.sleep(random.randint(1, 30))
                                print(f'{e}')
                            except openai.error.APIerror:
                                time.sleep(random.randint(1, 30))
                                print(f"API GG")
                            else:
                                success=True
                                break
                        if success:
                            res = response.choices[0].message.content
                            row.append(res)
                            
                    except:
                        pass

    all_data = thread_inputs
    pageQueue = Queue(len(all_data))
    for p in all_data:
        pageQueue.put(p)
    print(pageQueue.qsize())

    crawl_threads = []
    crawl_name_list = range(50)
    for thread_id in crawl_name_list:
        thread = Crawl_thread(thread_id, pageQueue)
        time.sleep(0.5)
        thread.start()
        crawl_threads.append(thread)
    for thread in crawl_threads:
        thread.join() 
    return [_[-1] for _ in thread_inputs]

In [254]:
#call api
prompts_outputs = api(prompts)
new_content = []
for case, x in zip(prompts_outputs, benchmark_data_with_pool):
    x["self_"+str(budget)+"_judgment_output"] = case
    new_content.append(x)
#write_output(POOL_DATA_PATH+benchmark+"_"+prompt_template+"_NewPool_"+base_judge+".json", new_content)

519
start thread: 0
start thread: 1
start thread: 2
start thread: 3
start thread: 4
start thread: 5
start thread: 6
start thread: 7
start thread: 8
start thread: 9
start thread: 10
start thread: 11
start thread: 12
start thread: 13
start thread: 14
start thread: 15
start thread: 16
start thread: 17
start thread: 18
start thread: 19
start thread: 20
start thread: 21
start thread: 22
start thread: 23
start thread: 24
start thread: 25
start thread: 26
start thread: 27
start thread: 28
start thread: 29
start thread: 30
start thread: 31
start thread: 32
start thread: 33
start thread: 34
start thread: 35
start thread: 36
start thread: 37
start thread: 38
start thread: 39
start thread: 40
start thread: 41
start thread: 42
start thread: 43
start thread: 44
start thread: 45
start thread: 46
start thread: 47
start thread: 48
start thread: 49
quit thread: 35
quit thread: 5
quit thread: 30
quit thread: 19
quit thread: 38
quit thread: 37
quit thread: 11
quit thread: 6
quit thread: 41
quit thread: 1

In [4]:
def rewardbench_evaluation(new_content, budget):
    PARTITION=[358, 358+456, 358+456+740, 1431]
    VANILLA_SUBSETS = [0, 0, 0, 0]
    NEW_SUBSETS = [0, 0, 0, 0]
    def parttt(nid):
        if nid < PARTITION[0]:
            return 0
        elif nid < PARTITION[1] and nid > PARTITION[0]:
            return 1
        elif nid < PARTITION[2] and nid > PARTITION[1]:
            return 2
        else:
            return 3
    COUNT=0
    for case in new_content:
        #判断Flip16_judgment_output
        if case["judgment_output"] is None or case["remove_other_"+str(budget)+"_judgment_output"] is None:
            COUNT += 1
            continue
        subset = parttt(COUNT)
        if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
            VANILLA_SUBSETS[subset]+=1
        if ("[[A]]" in case["remove_other_"+str(budget)+"_judgment_output"] and case["label"] == 1) or ("[[B]]" in case["remove_other_"+str(budget)+"_judgment_output"] and case["label"] == 2):
            NEW_SUBSETS[subset] += 1
            if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
                pass
            else:
                print(case["instruction"])
                print(case["answer_a"][1]["content"])
                print(case["answer_b"][1]["content"])
                print(case["judgment_output"])
                print(case["remove_other_16_judgment_output"])
                print("-"*100)
        COUNT+=1
    print("VANILLA:", VANILLA_SUBSETS[0]/358, VANILLA_SUBSETS[1]/456, VANILLA_SUBSETS[2]/740, VANILLA_SUBSETS[3]/1431, sum(VANILLA_SUBSETS)/len(new_content))
    print("NEW:", NEW_SUBSETS[0]/358, NEW_SUBSETS[1]/456, NEW_SUBSETS[2]/740, NEW_SUBSETS[3]/1431,sum(NEW_SUBSETS)/len(new_content))
def helpsteer_evaluation(new_content, budget):
    VANILLA_SUBSETS = [0]
    NEW_SUBSETS = [0]
    COUNT=0
    for case in new_content:
        #判断Flip
        sum_1, sum_2 = 0, 0
        for a,b in zip(case["external_info"][0], [0.65, 0.8, 0.45, 0.55, -0.4]):
            sum_1+=a*b
        for a,b in zip(case["external_info"][1], [0.65, 0.8, 0.45, 0.55, -0.4]):
            sum_2+=a*b
        if sum_1> sum_2:
            case["label"] = 1
        elif sum_1< sum_2:
            case["label"] = 2
        else:
            continue
        if case["judgment_output"] is None or case["self_"+str(budget)+"_judgment_output"] is None:
            COUNT += 1
            continue
        if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
            VANILLA_SUBSETS[0]+=1
            if ("[[A]]" in case["judgment_output"] and "[[B]]" in case["self_"+str(budget)+"_judgment_output"]) or ("[[B]]" in case["judgment_output"] and "[[A]]" in case["self_"+str(budget)+"_judgment_output"]):
                print("Instruction", case["instruction"])
                print("Judgment", case["judgment_output"])
                print("New_Judgment", case["self_"+str(budget)+"_judgment_output"])
        if ("[[A]]" in case["self_"+str(budget)+"_judgment_output"] and case["label"] == 1) or ("[[B]]" in case["self_"+str(budget)+"_judgment_output"] and case["label"] == 2):
            NEW_SUBSETS[0] += 1
            
        COUNT+=1
    print("VANILLA:", VANILLA_SUBSETS[0]/COUNT, COUNT)
    print("NEW:", NEW_SUBSETS[0]/COUNT, COUNT)
def mtbench_human_evaluation(new_content, budget):
    VANILLA_SUBSETS = [0]
    NEW_SUBSETS = [0]
    COUNT=0
    for case in new_content:
        #判断Flip
        
        if case["judgment_output"] is None or case["self_"+str(budget)+"_judgment_output"] is None:
            COUNT += 1
            continue
        if case["label"] == 3:
            continue
        if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
            VANILLA_SUBSETS[0]+=1
        if ("[[A]]" in case["self_"+str(budget)+"_judgment_output"] and case["label"] == 1) or ("[[B]]" in case["self_"+str(budget)+"_judgment_output"] and case["label"] == 2):
            NEW_SUBSETS[0] += 1
        COUNT+=1
    print("VANILLA:", VANILLA_SUBSETS[0]/COUNT, COUNT)
    print("NEW:", NEW_SUBSETS[0]/COUNT, COUNT)
def lfqa_evaluation(new_content, budget):
    VANILLA_SUBSETS = [0]
    NEW_SUBSETS = [0]
    COUNT=0
    for case in new_content:
        #判断Flip
        
        if case["judgment_output"] is None or case[str(budget)+"_judgment_output"] is None:
            COUNT += 1
            continue
        if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
            VANILLA_SUBSETS[0]+=1
        if ("[[A]]" in case[str(budget)+"_judgment_output"] and case["label"] == 1) or ("[[B]]" in case[str(budget)+"_judgment_output"] and case["label"] == 2):
            NEW_SUBSETS[0] += 1
        COUNT+=1
    print("VANILLA:", VANILLA_SUBSETS[0]/COUNT, COUNT)
    print("NEW:", NEW_SUBSETS[0]/COUNT, COUNT)
def judgebench_evaluation(new_content, budget):
    VANILLA_SUBSETS = [0]
    NEW_SUBSETS = [0]
    COUNT=0
    for case in new_content:
        #判断Flip
        if case["judgment_output"] is None or case["self_"+str(budget)+"_judgment_output"] is None:
            COUNT += 1
            continue
        if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
            VANILLA_SUBSETS[0]+=1
        if ("[[A]]" in case["self_"+str(budget)+"_judgment_output"] and case["label"] == 1) or ("[[B]]" in case["self_"+str(budget)+"_judgment_output"] and case["label"] == 2):
            NEW_SUBSETS[0] += 1
        COUNT+=1
    print("VANILLA:", VANILLA_SUBSETS[0]/COUNT, COUNT)
    print("NEW:", NEW_SUBSETS[0]/COUNT, COUNT)
def evalbias_evaluation(new_content, budget):
    VANILLA_SUBSETS = [0]
    NEW_SUBSETS = [0]
    COUNT=0
    for case in new_content:
        #判断Flip
        
        if case["judgment_output"] is None or case["remove_other_16_judgment_output"] is None:
            COUNT += 1
            continue
        if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
            VANILLA_SUBSETS[0]+=1
        if ("[[A]]" in case["remove_other_16_judgment_output"] and case["label"] == 1) or ("[[B]]" in case["remove_other_16_judgment_output"] and case["label"] == 2):
            NEW_SUBSETS[0] += 1
            if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
                pass
            else:
                print(case["instruction"])
                print(case["answer_a"][1]["content"])
                print(case["answer_b"][1]["content"])
                print(case["judgment_output"])
                print(case["remove_other_16_judgment_output"])
                print("-"*100)
        COUNT+=1
    print("VANILLA:", VANILLA_SUBSETS[0]/COUNT, COUNT)
    print("NEW:", NEW_SUBSETS[0]/COUNT, COUNT)
def alpacaeval_evaluation(new_content, budget):
    VANILLA_SUBSETS = [0]
    NEW_SUBSETS = [0]
    COUNT=0
    for case in new_content:
        #判断Flip
        
        if case["judgment_output"] is None or case[str(budget)+"_judgment_output"] is None:
            COUNT += 1
            continue
        if ("[[A]]" in case["judgment_output"] and case["label"] == 1) or ("[[B]]" in case["judgment_output"] and case["label"] == 2):
            VANILLA_SUBSETS[0]+=1
        if ("[[A]]" in case[str(budget)+"_judgment_output"] and case["label"] == 1) or ("[[B]]" in case[str(budget)+"_judgment_output"] and case["label"] == 2):
            NEW_SUBSETS[0] += 1
        COUNT+=1
    print("VANILLA:", VANILLA_SUBSETS[0]/COUNT, COUNT)
    print("NEW:", NEW_SUBSETS[0]/COUNT, COUNT)

In [5]:
new_content = benchmark_data_with_pool
if benchmark == "rewardbench":
    rewardbench_evaluation(new_content, budget=budget)
elif benchmark == "helpsteer2":
    helpsteer_evaluation(new_content, budget=budget)
elif benchmark == "mtbench_human":
    mtbench_human_evaluation(new_content, budget=budget)
elif benchmark == "lfqa":
    lfqa_evaluation(new_content, budget)
elif benchmark == "judgebench":
    judgebench_evaluation(new_content, budget)
elif benchmark == "evalbias":
    evalbias_evaluation(new_content, budget)

Generate a title for a story related to the following text. In 2018, the most recent year for which data are available as of 2021, the Centers for Disease Control and Prevention's (CDC) National Center for Health Statistics reports 38,390 deaths by firearm, of which 24,432 were by suicide. The rate of firearm deaths per 100,000 people rose from 10.3 per 100,000 in 1999 to 12 per 100,000 in 2017, with 109 people dying per day or about 14,542 homicides in total, being 11.9 per 100,000 in 2018.
"Echoes of Despair: The Silent Rise of America's Crisis"
"The Rising Toll: A Story of Gun Violence in America"
Both Assistant A and Assistant B provided titles for a story related to the text about firearm deaths in the United States. 

Assistant A's title, "Echoes of Despair: The Silent Rise of America's Crisis," captures the somber and serious nature of the statistics, emphasizing the emotional impact ("Echoes of Despair") and the gradual increase in the problem ("The Silent Rise"). It suggests a